In [1]:
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.linalg as sla
import pickle
import os
from sklearn.datasets import load_svmlight_file

from methods import GradientDescent, Newton, Nesterov, RBFGS, LBFGS, BFGS
from methods.rbfgs import Uniform, Gaussian, CustomDiscrete, ConstantDistribution, \
                          MatrixDistribution
from oracles import LogRegL2Oracle, create_log_reg_oracle, QuadraticOracle
from utils import select_basis_columns, run_rbfgs_experiment, run_all_methods

import scipy.sparse.linalg as spla
import scipy.sparse as sps

In [2]:
file = './datasets/colon-cancer'
A, b = load_svmlight_file(file, n_features=None)
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))
A = A.toarray()

(n, d) = (62, 2000)
density = 1.0


In [3]:
with open('./results/colon_cancer/regcoef=1.00e-04/svd.pkl', 'rb') as file:
    U, sigma_diag, V = pickle.load(file)
print('Smoothness based on singular values: {}'.format(sigma_diag.max()**2 / (4. * A.shape[0])))
print('Smoothness based on row norms: {}'.format(np.max(npla.norm(A, axis=1))**2 / 4.))

Smoothness based on singular values: 77.71678149773064
Smoothness based on row norms: 961.291553805779


In [6]:
sketch_sizes = [1, 10, 20, 50, 100, 200, 500]
def experiment_colon_cancer(regcoef):
    oracle = create_log_reg_oracle(A, b, regcoef)
    run_all_methods(
        oracle=oracle, 
        sketch_sizes=sketch_sizes, 
        max_iter=1000, 
        mat=A, 
        output_folder='./results/colon_cancer/regcoef={:.2e}'.format(regcoef), 
        sigma_tolerance=1e-8, 
        method_tolerance=1e-15, 
        stopping_criteria='grad_abs', 
        random_state=0, 
    )

In [8]:
%%time
experiment_colon_cancer(1e-4)

Read SVD from ./results/colon_cancer/regcoef=1.00e-04/svd.pkl
Singular values above tolerance: 62

RBFGS-SVD sketch... Done
RBFGS-SVD sketch no sigma... Done
RBFGS-gauss... Done
RBFGS-coord...Done
BFGS... Done
Nesterov...Done

All runs completed.
CPU times: user 57min 41s, sys: 2min 58s, total: 1h 40s
Wall time: 1min 54s


In [7]:
%%time
experiment_colon_cancer(1e-1)

Read SVD from ./results/colon_cancer/regcoef=1.00e-01/svd.pkl
Singular values above tolerance: 62

RBFGS-SVD sketch... Done
RBFGS-SVD sketch no sigma... Done
RBFGS-gauss... Done
RBFGS-coord...Done
BFGS... Done

All runs completed.
CPU times: user 1h 17min 38s, sys: 4min 45s, total: 1h 22min 24s
Wall time: 2min 34s


### Nesterov

In [4]:
file = './datasets/colon-cancer'
A, b = load_svmlight_file(file, n_features=None)
regcoef = 1e-1
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))
oracle = create_log_reg_oracle(A, b, regcoef=regcoef)

(n, d) = (62, 2000)
density = 1.0


In [5]:
np.random.seed(42)
x_0 = np.random.normal(size=A.shape[1])
output_folder = './results/colon_cancer/regcoef={:.2e}'.format(regcoef)
method = Nesterov(oracle, x_0, stopping_criteria='grad_abs', 
                  tolerance=1e-16)
method.run(max_iter=50000, max_time=60)
method.oracle = None
with open('{}/nesterov.pkl'.format(output_folder), 'wb') as file:
    pickle.dump(method, file)